## This codework is one of the project of the course "Machine Leaning for NLP3" of University of Paris Cité. The strutre of the code was given by professor of the course, M. Bernard Timothé. I and my teammate, Irene completed the some parts between lines of hashtags (##########) including essential part of training and prediciton of the model.

Goal
==
We are about to train a *sequence-to-sequence model* to predict a paragraph of Gustave Flaubert's *Madame Bovary* given the preceding paragraph.
The model (at least in its first version) does not use words as units of text but characters.

*   The encoder part, based on a bidirectional LSTM, reads an input paragraph and turns it into a set of tensors that serves as initial state for the decoder part.
*   The decoder part is based on an (unidirectional) LSTM. The state of the LSTM is used to compute a probability distribution over the alphabet (including space and punctuation marks) and is updated each time a character is predicted by the LSTM reading this character's embedding.
*   The goal is to get the best model. It is part of the job to define what this means. It is also part of the job to explain me how you get your best model.

In [ ]:
import torch

In [ ]:
!pip install nltk

Downloading the dataset
==
The dataset we are going to use is there: "https://www.gutenberg.org/cache/epub/14155/pg14155.txt"

We have to pre-process it a little bit in order to remove everything that is not part of the text and to split the actual text into paragraphs.

## toy data set

In [ ]:
use_toy_dataset = False # If True, a toy dataset (see below) is use instead of the real one.
# use_toy_dataset = True # If True, a toy dataset (see below) is use instead of the real one.

In [ ]:
# Downloads the dataset.
import urllib

tmp = urllib.request.urlretrieve("https://www.gutenberg.org/cache/epub/14155/pg14155.txt")
filename = tmp[0]
print(filename)

/tmp/tmptnonkf6c


In [ ]:
## Downloads the dataset.
another_file = urllib.request.urlretrieve("https://www.gutenberg.org/cache/epub/14157/pg14157.txt")
another_filename = another_file[0]
print(another_filename)

/tmp/tmpylz04f_4


In [ ]:
# Prints the first 200 lines in the file with their line number.
# This shows that we have a little bit of preprocessing to do in order to clean the data.
with open(filename) as f:
  for i in range(200):
    print(f"[{i}] {f.readline()}", end='')

In [ ]:
import re # Regular expression library
roman_regex = re.compile('^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$') # This regular expression matches Roman numerals but also the empty string.

EOP = '\n' # The end-of-line character will be used to mark the end of paragraphs.


with open(filename) as f:
  # We want to skip everything before the actual text of the novel.
  # The line "PREMIÈRE PARTIE" appears twice: in the table of content and then at the start of the first part of the actual text.
  # The following lines discard everything up to this second occurence (included).
  skip = 2
  while(skip > 0):
    line = f.readline().strip()
    if(line == "PREMIÈRE PARTIE"): skip -= 1;

  paragraphs = [] # Note that each dialog line will be considered a separate paragraph.
  paragraph_buffer = [] # List[str]; each element corresponds to a line in the original text file + an additonal space if necessary.
  while(True):
    line = f.readline().strip()
    if("END OF THE PROJECT GUTENBERG EBOOK MADAME BOVARY" in line): break # End of the actual text.

    if(line == ""): # We've reached the end of a paragraph.
      if(len(paragraph_buffer) > 0):
        paragraph_buffer.append(EOP) # End of the paragraph.

        paragraph = "".join(paragraph_buffer) # The different lines that make up the paragraph are joined into a single string.
        paragraphs.append(paragraph)
        paragraph_buffer = []
      continue

    if(roman_regex.match(line)): continue # Ignores the lines that indicate the beginning of a chapter.
    if(line.endswith(" PARTIE")): continue # Ignores the lines that indicate the beginning of a part.

    if((len(paragraph_buffer) > 0) and (paragraph_buffer[-1][-1] != '-')):
      paragraph_buffer.append(' ')
      '''Adds a space between consecutive lines except when the first one ends with "-"
       (e.g. if the word "pomme-de-terre" is split with "pomme-de-" at the end of a line and "terre" at the beginning of the next,
        we do not want to join the two lines with a space).'''
    paragraph_buffer.append(line)

paragraphs1 = paragraphs
# print(f"{len(paragraphs)} paragraphs read.")
# for i in range(3): print(paragraphs[i], end='')

In [ ]:
import re # Regular expression library
roman_regex = re.compile('^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$') # This regular expression matches Roman numerals but also the empty string.

EOP = '\n' # The end-of-line character will be used to mark the end of paragraphs.


with open(another_filename) as f:
  # We want to skip everything before the actual text of the novel.
  # The line "PREMIÈRE PARTIE" appears twice: in the table of content and then at the start of the first part of the actual text.
  # The following lines discard everything up to this second occurence (included).
  skip = 2
  while(skip > 0):
    line = f.readline().strip()
    if(line == "CHAPITRE I"): skip -= 1;


  paragraphs2 = [] # Note that each dialog line will be considered a separate paragraph.
  paragraph_buffer2 = [] # List[str]; each element corresponds to a line in the original text file + an additonal space if necessary.
  while(True):
    line = f.readline().strip()
    if("END OF THE PROJECT GUTENBERG EBOOK BOUVARD ET PÉCUCHET" in line): break # End of the actual text.

    if(line == ""): # We've reached the end of a paragraph.
      if(len(paragraph_buffer2) > 0):
        paragraph_buffer2.append(EOP) # End of the paragraph.

        paragraph = "".join(paragraph_buffer2) # The different lines that make up the paragraph are joined into a single string.
        paragraphs2.append(paragraph)
        paragraph_buffer2 = []
      continue

    if(roman_regex.match(line)): continue # Ignores the lines that indicate the beginning of a chapter.
    if(line.endswith(" PARTIE")): continue # Ignores the lines that indicate the beginning of a part.

    if((len(paragraph_buffer2) > 0) and (paragraph_buffer2[-1][-1] != '-')):
      paragraph_buffer2.append(' ')
      '''Adds a space between consecutive lines except when the first one ends with "-"
       (e.g. if the word "pomme-de-terre" is split with "pomme-de-" at the end of a line and "terre" at the beginning of the next,
        we do not want to join the two lines with a space).'''
    paragraph_buffer2.append(line)

# print(f"{len(paragraphs2)} paragraphs read.")
# for i in range(30): print(paragraphs2[i], end='')

In [ ]:
paragraphs = paragraphs1 + paragraphs2
len(paragraphs)

6278

## split

In [ ]:
# import random
# random.shuffle(paragraphs)
# dataset = {"train":[], "dev":[], "test":[]}
# dataset["train"] = paragraphs[:(int(len(paragraphs)*0.7))]
# dataset["dev"] = paragraphs[int(len(paragraphs)*0.7):int(len(paragraphs)*0.9)]
# dataset["test"] = paragraphs[int(len(paragraphs)*0.9):]
# for part in dataset.keys(): print(len(dataset[part]))

Here, we define a toy dataset on which your model, if correctly implemented, should be able to learn more easily.

In [ ]:
# use_toy_dataset = False

In [ ]:
if(use_toy_dataset):
  paragraphs = []

  import random, string
  characters = list(string.ascii_lowercase + string.ascii_lowercase.upper() + "_-/\'[]()")
  # print("characters", characters) #len 60
  random.shuffle(characters)
  k = random.randint(1, 10)
  a = "a"
  paragraph = (a * k)
  for _ in range(100):
    random.shuffle(characters)
    for a in characters:
      k = random.randint(1, 16)
      paragraph += f"? Now, please write {k} {a}.{EOP}"
      paragraphs.append(paragraph)
      paragraph = (a * k)
  print(f"{len(paragraphs)} paragraphs generated.")

  print(paragraphs[:10])

In [ ]:
import collections
import numpy as np

# Computes the frequency of all characters in the dataset.
char_counts = collections.defaultdict(int)
for paragraph in paragraphs:
  for char in paragraph: char_counts[char] += 1

print(f"{len(char_counts)} different characters found in the dataset.") # 60 becomes 75 because of numbers added (randint of k)
print(sorted(char_counts.items(), key=(lambda x: x[1]), reverse=True)) # Shows each character with its frequency, in decreasing frequency order.

100 different characters found in the dataset.
[(' ', 191850), ('e', 137585), ('a', 78879), ('s', 76996), ('t', 69896), ('i', 68086), ('n', 64638), ('r', 62894), ('u', 59983), ('l', 58562), ('o', 49667), ('d', 35130), ('c', 27866), ('m', 25786), ('p', 24691), (',', 20222), ('é', 15719), ('v', 15206), ("'", 12853), ('.', 11559), ('b', 9909), ('f', 9776), ('h', 9723), ('-', 9216), ('q', 8913), ('g', 8868), ('\n', 6278), ('à', 4273), ('x', 4010), ('è', 3538), ('j', 2906), ('!', 2901), ('y', 2896), (';', 2317), ('L', 2188), ('ê', 2109), ('E', 2090), ('C', 1750), ('P', 1611), ('I', 1589), ('M', 1544), ('B', 1405), ('z', 1159), ('A', 1062), ('?', 1007), ('D', 862), (':', 853), ('â', 779), ('ç', 749), ('S', 691), ('î', 583), ('O', 574), ('R', 515), ('V', 497), ('ô', 493), ('J', 485), ('T', 484), ('Q', 455), ('ù', 430), ('H', 415), ('û', 388), ('U', 347), ('N', 338), ('F', 327), ('G', 292), ('_', 160), ('À', 155), ('«', 126), ('É', 116), ('»', 115), ('ï', 90), ('Y', 86), ('(', 82), (')', 82), 

In [ ]:
# Here you have to build a dictionary 'char_vocabulary' that assigns an integer id to each character,
# along with a list/array 'id_to_char' that implements the reverse mapping.
#################
id_to_char = list(char_counts.keys())
char_vocabulary = { ch : idx for idx, ch in list(enumerate(id_to_char))}
print("id_to_char", id_to_char)
print("char_vocabulary", char_vocabulary)
#################

id_to_char ['N', 'o', 'u', 's', ' ', 'é', 't', 'i', 'n', 'à', 'l', "'", 'É', 'd', 'e', ',', 'q', 'a', 'P', 'r', 'v', 'h', 'b', 'g', 'ç', 'c', 'p', '.', 'C', 'x', 'm', 'è', '\n', 'L', 'f', ';', 'î', ':', '-', 'M', 'R', 'j', 'S', 'ù', 'â', 'z', 'I', 'Q', 'û', 'ê', 'O', 'y', 'k', 'ï', 'E', 'T', 'U', 'D', '!', '(', ')', 'B', '?', '_', 'G', '1', '8', '2', 'A', 'À', 'H', 'ô', 'V', 'Ê', '«', '»', 'Y', 'F', 'J', 'ë', 'W', 'X', '0', '5', '7', '9', '6', '3', 'w', 'Î', 'Ç', '4', 'ü', '°', 'K', 'Z', 'È', '=', 'ñ', 'º']
char_vocabulary {'N': 0, 'o': 1, 'u': 2, 's': 3, ' ': 4, 'é': 5, 't': 6, 'i': 7, 'n': 8, 'à': 9, 'l': 10, "'": 11, 'É': 12, 'd': 13, 'e': 14, ',': 15, 'q': 16, 'a': 17, 'P': 18, 'r': 19, 'v': 20, 'h': 21, 'b': 22, 'g': 23, 'ç': 24, 'c': 25, 'p': 26, '.': 27, 'C': 28, 'x': 29, 'm': 30, 'è': 31, '\n': 32, 'L': 33, 'f': 34, ';': 35, 'î': 36, ':': 37, '-': 38, 'M': 39, 'R': 40, 'j': 41, 'S': 42, 'ù': 43, 'â': 44, 'z': 45, 'I': 46, 'Q': 47, 'û': 48, 'ê': 49, 'O': 50, 'y': 51, 'k': 52, 'ï

In [ ]:
EOP_id = char_vocabulary[EOP] # Id for the end-of-paragraph symbol

print(char_vocabulary)
print(id_to_char)
print(f"EOP_id = {EOP_id}")

# Here you have to implement a test that proves that your implementations of 'char_vocabulary' and 'id_to_char' are consistent.
#################
# iterating on char is is more elegant and solid way to test
for ch in id_to_char:
  assert char_vocabulary[ch] == char_vocabulary[id_to_char[char_vocabulary[ch]]]
print("Every mapping of char and id is consistent.")


#################

{'N': 0, 'o': 1, 'u': 2, 's': 3, ' ': 4, 'é': 5, 't': 6, 'i': 7, 'n': 8, 'à': 9, 'l': 10, "'": 11, 'É': 12, 'd': 13, 'e': 14, ',': 15, 'q': 16, 'a': 17, 'P': 18, 'r': 19, 'v': 20, 'h': 21, 'b': 22, 'g': 23, 'ç': 24, 'c': 25, 'p': 26, '.': 27, 'C': 28, 'x': 29, 'm': 30, 'è': 31, '\n': 32, 'L': 33, 'f': 34, ';': 35, 'î': 36, ':': 37, '-': 38, 'M': 39, 'R': 40, 'j': 41, 'S': 42, 'ù': 43, 'â': 44, 'z': 45, 'I': 46, 'Q': 47, 'û': 48, 'ê': 49, 'O': 50, 'y': 51, 'k': 52, 'ï': 53, 'E': 54, 'T': 55, 'U': 56, 'D': 57, '!': 58, '(': 59, ')': 60, 'B': 61, '?': 62, '_': 63, 'G': 64, '1': 65, '8': 66, '2': 67, 'A': 68, 'À': 69, 'H': 70, 'ô': 71, 'V': 72, 'Ê': 73, '«': 74, '»': 75, 'Y': 76, 'F': 77, 'J': 78, 'ë': 79, 'W': 80, 'X': 81, '0': 82, '5': 83, '7': 84, '9': 85, '6': 86, '3': 87, 'w': 88, 'Î': 89, 'Ç': 90, '4': 91, 'ü': 92, '°': 93, 'K': 94, 'Z': 95, 'È': 96, '=': 97, 'ñ': 98, 'º': 99}
['N', 'o', 'u', 's', ' ', 'é', 't', 'i', 'n', 'à', 'l', "'", 'É', 'd', 'e', ',', 'q', 'a', 'P', 'r', 'v', 'h

In [ ]:
# Turns a list of lists of ids into a list of strings.
# Do not forget that an occurrence of EOP means that the paragraph ends here.
def ids_to_texts(ids):
  # Here you have to turn each list of character ids of 'ids' into a string and then return all strings as a list.
  #################
  res = ""
  fin = []
  for para_id in ids :
    for char_id in para_id :
      if char_id != char_vocabulary[EOP]:
        res += id_to_char[char_id]
      else:
        break
    fin.append(res)
    res = ""
  return fin

  #################

In [ ]:
ps = ["Bonjour.", "Comment allez vous ?"]
ids = [[char_vocabulary[c] for c in p] for p in ps]
print("ids : ", ids)
print( "ids_to_texts : ", ids_to_texts(ids))
print(f"'ids_to_texts(ids) == ps' should be True: {ids_to_texts(ids) == ps}")

ids :  [[61, 1, 8, 41, 1, 2, 19, 27], [28, 1, 30, 30, 14, 8, 6, 4, 17, 10, 10, 14, 45, 4, 20, 1, 2, 3, 4, 62]]
ids_to_texts :  ['Bonjour.', 'Comment allez vous ?']
'ids_to_texts(ids) == ps' should be True: True


In [ ]:
ps = ["Bonjour.", "Comment allez vous ?"]
ids = [[char_vocabulary[c] for c in p] for p in ps]
print(ids)
ids[0].extend([EOP_id, (EOP_id+1), (EOP_id+1)]) # With the end-of-paragraph token id and additional (padding-like) stuff for the first string.
print(ids)
print(ids_to_texts(ids))
print(f"'ids_to_texts(ids) == ps' should be True: {ids_to_texts(ids) == ps}")
# If you have a problem here, remember that EOP indicates the end of the text (this might be related to your problem).

[[61, 1, 8, 41, 1, 2, 19, 27], [28, 1, 30, 30, 14, 8, 6, 4, 17, 10, 10, 14, 45, 4, 20, 1, 2, 3, 4, 62]]
[[61, 1, 8, 41, 1, 2, 19, 27, 32, 33, 33], [28, 1, 30, 30, 14, 8, 6, 4, 17, 10, 10, 14, 45, 4, 20, 1, 2, 3, 4, 62]]
['Bonjour.', 'Comment allez vous ?']
'ids_to_texts(ids) == ps' should be True: True


Batch generator
==

In [ ]:
# Defines a class of objects that produce batches from the dataset.
# A training instance is composed of a pair of consecutive paragraphs.
# The goal will be to predict the second given the first.
# (Possible improvement: As is, ends of chapter are completely ignored:
# the last paragraph of a chapter and the first of the following chapter form a training instance.
# We might want to predict the end of the chapter instead, or simply remove these pairs from the dataset.)

class BatchGenerator:
  def __init__(self, paragraphs, char_vocabulary):
    self.paragraphs = paragraphs
    self.char_vocabulary = char_vocabulary # Dictionary
    self.padding_idx = len(char_vocabulary)

  # Returns the number of training instances (i.e. of pairs of consecutive paragraphs).
  def length(self):
    return (len(self.paragraphs) - 1)

  # Returns a random training batch (composed of pairs of consecutive paragraphs).
  # If `subset` is an integer, only a subset of the corpus is used. This can be useful when debugging the system.
  def get_batch(self, batch_size, subset=None):
    max_i = self.length() if(subset is None) else min(subset, self.length())
    paragraph_ids = np.random.randint(max_i, size=batch_size) # Randomly picks some paragraph ids.

    return self._ids_to_batch(paragraph_ids)

  def _ids_to_batch(self, paragraph_ids):
    firsts = [] # First paragraph of each pair
    seconds = [] # Second paragraph of each pair
    for paragraph_id in paragraph_ids:
      firsts.append([self.char_vocabulary[char] for char in self.paragraphs[paragraph_id]])
      seconds.append([self.char_vocabulary[char] for char in self.paragraphs[paragraph_id + 1]])

    # Padding
    self.pad(firsts)
    self.pad(seconds)

    firsts = torch.tensor(firsts, dtype=torch.long) # Conversion to a tensor
    seconds = torch.tensor(seconds, dtype=torch.long) # Conversion to a tensor

    return (firsts, seconds)

  # Pads a list of lists (i.e. adds fake word ids so that all sequences in the batch have the same length, so that we can use a matrix to represent them).
  # In place
  def pad(self, sequences):
    max_length = max([len(s) for s in sequences])
    for s in sequences: s.extend([self.padding_idx] * (max_length - len(s)))

  # Returns a generator of training batches for a full epoch.
  #  (Note that this function is not used in the training loop implemented below. `get_batch` is used instead.)
  # If `subset` is an integer, only a subset of the corpus is used. This can be useful when debugging the system.
  def all_batches(self, batch_size, subset=None):
    max_i = self.length() if(subset is None) else min(subset, self.length())

    # Loop that generates all full batches (batches of size 'batch_size').
    i = 0
    while((i + batch_size) <= max_i):
      instance_ids = np.arange(i, (i + batch_size)) #numbers from i to (i+bsize)
      yield self._ids_to_batch(instance_ids)
      i += batch_size

    # Possibly generates the last (not full) batch.
    if(i < max_i):
      instance_ids = np.arange(i, max_i)
      yield self._ids_to_batch(instance_ids)

  # Turns a list of arbitrary paragraphs into a prediction batch.
  def turn_into_batch(self, paragraphs):
    firsts = []
    for paragraph in paragraphs:
        # Unknown characters are ignored (removed).
        tmp = []
        for char in paragraph:
          if(char in self.char_vocabulary): tmp.append(self.char_vocabulary[char])

        if(tmp[-1] != EOP_id): tmp.append(EOP_id) # Adds an end-of-paragraph character if necessary.

        firsts.append(tmp)

    self.pad(firsts)
    return torch.tensor(firsts, dtype=torch.long)

batch_generator = BatchGenerator(paragraphs=paragraphs, char_vocabulary=char_vocabulary)
# batch_generator = BatchGenerator(paragraphs=['Bonjour.\n', 'Salutoioiiiii\n', 'Hello\n', 'World\n', 'Bye\n', 'Thanks\n'], char_vocabulary=char_vocabulary)
# train_batch_generator = BatchGenerator(paragraphs=dataset["train"], char_vocabulary=char_vocabulary)

print(batch_generator.length())

6277


In [ ]:
# batch_generator.paragraphs
# batch_generator.padding_idx

In [ ]:
(firsts, seconds) = batch_generator.get_batch(3)
print(ids_to_texts(firsts))
print(ids_to_texts(seconds))

["-- J'ai été chez trois personnes... inutilement!", 'Il reparut avec elle, et un Molière.', "Le notaire l'arrêta; et se haussant à chaque mot sur la pointe des orteils:"]
["Puis ils restèrent assis l'un en face de l'autre, aux deux coins de la cheminée, immobiles, sans parler. Emma haussait les épaules, tout en trépignant. Il l'entendit qui murmurait:", "Le commencement fut médiocre. Mais Tartuffe venant à caresser les genoux d'Elmire, Pécuchet prit un ton de gendarme.", "--Je trouve votre système d'une immoralité complète. Il donne carrière à tous les débordements, excuse les crimes, innocente les coupables."]


In [ ]:
test = ids_to_texts(firsts)
print("test :", test)
batch = batch_generator.turn_into_batch(test)
print("batch : ", batch) # (B, seq_len)
text = ids_to_texts(batch)
print("back to text : ", text)

test : ["-- J'ai été chez trois personnes... inutilement!", 'Il reparut avec elle, et un Molière.', "Le notaire l'arrêta; et se haussant à chaque mot sur la pointe des orteils:"]
batch :  tensor([[ 38,  38,   4,  78,  11,  17,   7,   4,   5,   6,   5,   4,  25,  21,
          14,  45,   4,   6,  19,   1,   7,   3,   4,  26,  14,  19,   3,   1,
           8,   8,  14,   3,  27,  27,  27,   4,   7,   8,   2,   6,   7,  10,
          14,  30,  14,   8,   6,  58,  32, 100, 100, 100, 100, 100, 100, 100,
         100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
         100, 100, 100, 100, 100, 100],
        [ 46,  10,   4,  19,  14,  26,  17,  19,   2,   6,   4,  17,  20,  14,
          25,   4,  14,  10,  10,  14,  15,   4,  14,   6,   4,   2,   8,   4,
          39,   1,  10,   7,  31,  19,  14,  27,  32, 100, 100, 100, 100, 100,
         100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
         100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 10

The model
==
For this model, we will not define a `forward` method, but two methods: `trainingLogits` and `predictionStrings`.

*    `trainingLogits` is used at training time, when each batch is split in two parts: input paragraphs and output paragraphs. This function outputs, for each output paragraph of the batch, a log-probability distribution (i.e. a vector of "logits") before each token and after the last one. These distributions depend on the encoding of the corresponding input paragraph. They will then be used to compute a loss value.
*    `predictionStrings` is used at prediction time, when each batch is only composed of input paragraphs. This function outputs, for each input paragraph, a string obtained by decoding the encoding of the paragraph.

(Don't forget to read carefully all comments and to make sure that you understand them.)

Here is a graphical representation of the architecture: https://moodle.u-paris.fr/mod/resource/view.php?id=648001
Before starting the implementation, make sure you understand it.

In [ ]:
class Model(torch.nn.Module):
  # 'size_vocabulary' does not include a padding character, but does include the end-of-paragraph one.
  def __init__(self, size_vocabulary, EOP_id, embedding_dim, lstm_hidden_size, lstm_layers, device='cpu'):
    super().__init__()

    self.device = device

    self.EOP_id = EOP_id # At prediction time, this index is used to stop the generation at the end of the paragraph.

    # Here you have to define:
    #  (i) an embedding layer 'self.char_embeddings' with 'torch.nn.Embedding' for the characters, including an padding embedding;
    #  (ii) a bidirectional LSTM 'self.encoder_lstm' with a hidden size of 'lstm_hidden_size' and 'lstm_layers' layers (use batch_first=True);
    #  (iii) a unidirectional LSTM 'self.decoder_lstm' with a hidden size of 'lstm_hidden_size' and 'lstm_layers' layers (use batch_first=True);
    #  (iv) a network 'self.decoder_initialiser' meant to turn the final hidden and cell states of the encoder into the initial hidden and cell states of the decoder;
    #  (v) a network 'self.distribution_nn' meant to turn the hidden state of the decoder at each step into the logits of a probability distribution over the vocabulary.
        # The logits of a probability distribution are simply the log-probabilities (you might want to use torch.nn.LogSoftmax).
    # Send all parts to 'device', so that we can use a GPU.
    #################
    self.size_vocabulary =size_vocabulary; self.embedding_dim = embedding_dim
    self.lstm_hidden_size =lstm_hidden_size; self.lstm_layers =lstm_layers

    self.char_embeddings = torch.nn.Embedding(self.size_vocabulary+1, self.embedding_dim, padding_idx = self.size_vocabulary).to(device)
    self.encoder_lstm = torch.nn.LSTM(input_size = self.embedding_dim,
                                      hidden_size = self.lstm_hidden_size,
                                      ## Each LSTM cell has an interanl two vectors: the hidden state (h) and the cell state (c). The hidden_size refers to the dimensionality of these vectors.
                                      num_layers = self.lstm_layers, bidirectional = True, batch_first=True).to(device)
    self.decoder_lstm = torch.nn.LSTM(input_size = self.embedding_dim,
                                      hidden_size = self.lstm_hidden_size,
                                      num_layers = self.lstm_layers, bidirectional = False, batch_first=True).to(device) ## (bsize, seqlen, hidd)
    self.decoder_initialiser = torch.nn.Linear(self.lstm_hidden_size * 2, self.lstm_hidden_size).to(device)

    self.distribution_nn = torch.nn.Sequential(torch.nn.Linear(lstm_hidden_size, size_vocabulary),
                                               torch.nn.LogSoftmax(dim=-1)).to(device)

    # self.decoder_output = torch.nn.Linear(lstm_hidden_size, size_vocabulary).to(device)
    # softmax = torch.nn.LogSoftmax(dim=-1).to(device)
    # self.distribution_nn = softmax(self.decoder_output).to(device)


    #################

  # This function encodes the input paragraphs and turns them into initial states for the decoder. It is used both at training and prediction time.
  # 'in_paragraphs' is a matrix (batch size, max in length) of character ids (Integer).
  # You might want to understand what is the output of PyTorch's LSTMs: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
  def initStates(self, in_paragraphs):
    batch_size = in_paragraphs.size(0)

    in_char_embeddings = self.char_embeddings(in_paragraphs) # Shape: (batch_size, max length, embedding size)
    #print(in_char_embeddings); print(in_char_embeddings.shape)
    in_lengths = (in_paragraphs != self.char_embeddings.padding_idx).sum(axis=1) # Shape: (batch_size)
    #print(in_lengths); print(in_lengths.shape)
    in_char_embeddings = torch.nn.utils.rnn.pack_padded_sequence(input=in_char_embeddings, lengths=in_lengths.cpu(), batch_first=True, enforce_sorted=False)
    # Enables the biLSTM to ignore padding elements. ## will obtain sequences concatenated in a single tensor

    # The input paragraphs are encoded; the final hidden and cell states of the network will be used to initialise the decoder after a little transformation.
    _, (h_n, c_n) = self.encoder_lstm(in_char_embeddings) # 'h_n' and 'c_n' are both of shape (num_layers * 2, batch_size, hidden_size)

    # Concatenates the left-to-right and right-to-left final hidden states of the biLSTM.
    h_n = h_n.view(self.encoder_lstm.num_layers, 2, batch_size, self.encoder_lstm.hidden_size)
    # The second dimension (of size 2) of this tensor corresponds to left-to-right (0) and right-to-left (1).
    #print(h_n); print(h_n.shape)
    lr_h_n = h_n[:,0] # left-to-right; shape: (num_layers, batch_size, hidden_size)
    rl_h_n = h_n[:,1] # right-to-left; shape: (num_layers, batch_size, hidden_size)
    bi_h_n = torch.cat([lr_h_n, rl_h_n], axis=2) # Shape: (num_layers, batch_size, (2 * hidden_size))
    #print(bi_h_n); print(bi_h_n.shape)

    # Concatenates the left-to-right and right-to-left final cell states of the biLSTM.
    c_n = c_n.view(self.encoder_lstm.num_layers, 2, batch_size, self.encoder_lstm.hidden_size)
    # The second dimension (of size 2) of this tensor corresponds to left-to-right (0) and right-to-left (1).
    #print(c_n); print(c_n.shape)
    lr_c_n = c_n[:,0] # left-to-right; shape: (num_layers, batch_size, hidden_size)
    rl_c_n = c_n[:,1] # right-to-left; shape: (num_layers, batch_size, hidden_size)
    bi_c_n = torch.cat([lr_c_n, rl_c_n], axis=2) # Shape: (num_layers, batch_size, (2 * hidden_size))
    #print(bi_c_n); print(bi_c_n.shape)

    # What should be the shape of the two tensors of the following pair? Answer: (num_layers, batch_size, hidden_size)
    return (self.decoder_initialiser(bi_h_n), self.decoder_initialiser(bi_c_n))

  # Training time: This function outputs the logits for each time step.
  # Because at training time, the output paragraph is known, there is no need to generate anything sequentially
  #   — all positions can be processed at the same time. In fact, there is a loop hidden in the call to the decoder LSTM, but you should not write any explicit loop here.
  # Do not forget the distribution for the first character.
  # 'in_paragraphs' is a matrix (batch size, max in length) of character ids (Integer).
  # 'out_paragraphs' is a matrix (batch size, max out length) of character ids (Integer) at training time. Assume it does not include the final end-of-paragraph character.
  # You might want to understand what is the output of PyTorch's LSTMs: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html


  def trainingLogits(self, in_paragraphs, out_paragraphs):
    decoder_init_states = self.initStates(in_paragraphs)
    # These tensors are not only used to initialise the decoder but also (for the first tensor) to compute the probability distributions for the first character.

    # Feed a packed sequence to the decoder (use 'torch.nn.utils.rnn.pack_padded_sequence' and 'torch.nn.utils.rnn.pad_packed_sequence').
    # You don't need to implement a loop, because at training time, you know in advance the decisions of the system (i.e. the tokens that are generated).
    #################

    out_char_embeddings = self.char_embeddings(out_paragraphs) # Shape: (batch_size, max length, embedding size)
    #print(in_char_embeddings); print(in_char_embeddings.shape)
    out_lengths = (out_paragraphs != self.char_embeddings.padding_idx).sum(axis=1) # Shape: (batch_size)
    #print(in_lengths); print(in_lengths.shape)
    out_char_embeddings = torch.nn.utils.rnn.pack_padded_sequence(input=out_char_embeddings, lengths=out_lengths.cpu(), batch_first=True, enforce_sorted=False)
    # Enables the biLSTM to ignore padding elements. ## will obtain sequences are concatenated in a single tensor

    # The input paragraphs are encoded; the final hidden and cell states of the network will be used to initialise the decoder after a little transformation.
    '''EK done // double hash tag for my personal comment '''
    ## here we want to concat (A)final hidden state of initializer & (B) ouput of decoder

    ## (A)final hidden state of initializer
    fin_decoder_init_states = decoder_init_states[0][-1][:][:] ## size (batch, hidden)
    # print(fin_decoder_init_states.size())
    ## (B)ouput of decoder
    packed_out, (_, _) = self.decoder_lstm(out_char_embeddings, decoder_init_states) ## size of packed_output : vocab size?
    unpacked_out, unpacked_len = torch.nn.utils.rnn.pad_packed_sequence(packed_out, batch_first=True) ## size of unpacked_out : ( batch X max_seq_len X hidden_dim)
    # decoder_out = self.decoder_output(unpacked_out) # shape = (batch_size, max length, hidden size)
    # print("unpacked_out",unpacked_out.size())
    ## concat of A&B is  (batch, hidden) & (batch, max_seq_len, hidden); to concatenate, 2 dim to be matched which is batch and hid
    ## so lets make   (1, batch, hidden) & (max_seq_len, batch, hidden)
    fin_decoder_init_states = torch.unsqueeze(fin_decoder_init_states, 0) ## (1, batch, hidden)
    # print("fin_decoder_init_states - ", fin_decoder_init_states.size())

    permuted_out = torch.permute(unpacked_out,(1,0,2)) ## (max_seq_len, batch, hidden)
    # print("permuted_out - ", permuted_out.size())

    concatenated = torch.cat((fin_decoder_init_states, permuted_out)) ## (1+max_seq_len, batch, hidden)
    concatenated = torch.permute(concatenated,(1,0,2)) ## (B, 1+max_seq_len, hidden)
    final_proba = self.distribution_nn(concatenated) ## (B, 1+max_seq_len, size_voca)

    # print("final_proba - ", final_proba.size())
    return final_proba[:, :-1, :]

    # permuted = torch.permute(decoder_init_states[0],(1,0,2))[:][-1][:]
    # print(permuted.size())
    # print(unpacked_out.size())
    # concatenated = torch.cat((permuted, unpacked_out))


    #################

  # Prediction time: This function generates a text up to 'max_predicted_char' character long for each paragraph in the batch.
  # 'in_paragraphs' is a matrix (batch size, max in length) of character ids (Integer).
  # You might want to understand what is the output of PyTorch's LSTMs: https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html
  def predictionStrings(self, in_paragraphs, max_predicted_char=1000):
    batch_size = in_paragraphs.size(0)

    decoder_init_states = self.initStates(in_paragraphs) # These tensors are not only used to initialise the decoder but also (for the first tensor) to compute the probability distributions for the first character.

    # Decode 'decoder_init_states' into a matrix a character ids (on line per input paragraph in the batch) and then convert it to strings of actual characters.
    # You will need to implement a loop at some point.
    # To work with probability distributions, you may use "torch.distributions.Categorical", but not necessarily.
    #################

    hidden, cell = decoder_init_states
    final_hidden = hidden[-1][:][:] # (batch, hidden)
    final_hidden = final_hidden.unsqueeze(0) # (seq_len, batch, hidden)
    final_hidden = final_hidden.permute(1,0,2) # (batch, seq_len, hidden)



    proba_first_char = self.distribution_nn(final_hidden)  # (batch, seq_len, voca_size)
    first_char_id = torch.argmax(proba_first_char, dim =2) #(batch, seq_len)

    seq_ids = torch.clone(first_char_id) #(batch, seq_len)

    char_embedding = self.char_embeddings(first_char_id) #(batch, seq_len, emb_dim)

    for char in range(max_predicted_char):
     out, (hidden,cell) = self.decoder_lstm(char_embedding, (hidden, cell))
     proba_char = self.distribution_nn(out)
     char_id = torch.argmax(proba_char, dim =2)
    #  char_id = torch.distributions.Categorical(proba_char).sample()
    #  print(char_id)
    #  print(char_id[-1])
     seq_ids = torch.cat((seq_ids, char_id), 1)
     char_embedding = self.char_embeddings(char_id) #(batch, seq_len, emb_dim)

    texts = ids_to_texts(seq_ids)
    return texts



In [ ]:
model = Model(size_vocabulary=len(char_vocabulary), EOP_id=EOP_id, embedding_dim=19, lstm_hidden_size=13, lstm_layers=7, device='cuda')

In [ ]:
# Tests the training method (again).
in_paragraphs = torch.tensor([(list(range(5)) + ([batch_generator.padding_idx] * 10)), (list(range(10)) + ([batch_generator.padding_idx] * 5))]).to(model.device)
# A batch that contains two sentences with some padding (more than necessary).

print(in_paragraphs)
out_paragraphs = in_paragraphs
model.trainingLogits(in_paragraphs, out_paragraphs)

tensor([[  0,   1,   2,   3,   4, 100, 100, 100, 100, 100, 100, 100, 100, 100,
         100],
        [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 100, 100, 100, 100,
         100]], device='cuda:0')


tensor([[[-4.2358, -4.5068, -4.8697,  ..., -4.6520, -4.6366, -4.2372],
         [-4.3794, -4.5103, -4.8153,  ..., -4.6417, -4.6069, -4.4068],
         [-4.4052, -4.5052, -4.7866,  ..., -4.6241, -4.6057, -4.4560],
         ...,
         [-4.3816, -4.5838, -4.7885,  ..., -4.6088, -4.6250, -4.4122],
         [-4.3816, -4.5838, -4.7885,  ..., -4.6088, -4.6250, -4.4122],
         [-4.3816, -4.5838, -4.7885,  ..., -4.6088, -4.6250, -4.4122]],

        [[-4.2405, -4.5060, -4.8683,  ..., -4.6538, -4.6359, -4.2351],
         [-4.3819, -4.5099, -4.8145,  ..., -4.6428, -4.6066, -4.4056],
         [-4.4066, -4.5049, -4.7861,  ..., -4.6247, -4.6057, -4.4553],
         ...,
         [-4.4190, -4.4963, -4.7568,  ..., -4.5926, -4.6260, -4.4984],
         [-4.4181, -4.4961, -4.7563,  ..., -4.5908, -4.6278, -4.4993],
         [-4.4174, -4.4961, -4.7560,  ..., -4.5896, -4.6290, -4.4998]]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
# Tests the training method.
in_paragraphs = torch.tensor([(list(range(5)) + ([batch_generator.padding_idx] * 0))]).to(model.device)
# A batch that contains only one sentence with no padding.
print(in_paragraphs)
out_paragraphs = in_paragraphs
model.trainingLogits(in_paragraphs, out_paragraphs)

tensor([[0, 1, 2, 3, 4]], device='cuda:0')


tensor([[[-4.2358, -4.5068, -4.8697, -4.8595, -4.5959, -4.6761, -4.3968,
          -5.0175, -4.4828, -4.7382, -4.4449, -4.6436, -4.8952, -4.3654,
          -4.3695, -4.4616, -4.5954, -4.4299, -4.5288, -5.0503, -4.7347,
          -4.7167, -4.4404, -4.5566, -4.5392, -4.4191, -4.7795, -4.5659,
          -4.7698, -4.8030, -4.6554, -4.9666, -4.5925, -4.4994, -4.7906,
          -4.3801, -4.3415, -4.2707, -4.8580, -4.4736, -4.4353, -4.6041,
          -4.9328, -4.7458, -4.8218, -4.5234, -4.6623, -4.3564, -4.5188,
          -4.7601, -4.9271, -4.5766, -4.5568, -4.5569, -4.6334, -4.7228,
          -4.6722, -4.2496, -4.7030, -4.3837, -4.6527, -4.6390, -4.6827,
          -4.6894, -4.5634, -4.4429, -4.5277, -4.3899, -4.6573, -4.9238,
          -4.9086, -4.6949, -4.4053, -4.6746, -4.8806, -4.6918, -4.8593,
          -4.3735, -4.4694, -4.6711, -4.4048, -4.8286, -4.7878, -4.6702,
          -4.4909, -4.4820, -4.6091, -4.7716, -4.8147, -4.4582, -4.6196,
          -4.7002, -4.8647, -4.7711, -4.5218, -4.66

In [ ]:
# Tests the training method (again).
in_paragraphs = torch.tensor([(list(range(5)) + ([batch_generator.padding_idx] * 10)), (list(range(10)) + ([batch_generator.padding_idx] * 5))]).to(model.device)
# A batch that contains two sentences with some padding (more than necessary).
print(in_paragraphs)
out_paragraphs = in_paragraphs
model.trainingLogits(in_paragraphs, out_paragraphs)

tensor([[  0,   1,   2,   3,   4, 100, 100, 100, 100, 100, 100, 100, 100, 100,
         100],
        [  0,   1,   2,   3,   4,   5,   6,   7,   8,   9, 100, 100, 100, 100,
         100]], device='cuda:0')


tensor([[[-4.2358, -4.5068, -4.8697,  ..., -4.6520, -4.6366, -4.2372],
         [-4.3794, -4.5103, -4.8153,  ..., -4.6417, -4.6069, -4.4068],
         [-4.4052, -4.5052, -4.7866,  ..., -4.6241, -4.6057, -4.4560],
         ...,
         [-4.3816, -4.5838, -4.7885,  ..., -4.6088, -4.6250, -4.4122],
         [-4.3816, -4.5838, -4.7885,  ..., -4.6088, -4.6250, -4.4122],
         [-4.3816, -4.5838, -4.7885,  ..., -4.6088, -4.6250, -4.4122]],

        [[-4.2405, -4.5060, -4.8683,  ..., -4.6538, -4.6359, -4.2351],
         [-4.3819, -4.5099, -4.8145,  ..., -4.6428, -4.6066, -4.4056],
         [-4.4066, -4.5049, -4.7861,  ..., -4.6247, -4.6057, -4.4553],
         ...,
         [-4.4190, -4.4963, -4.7568,  ..., -4.5926, -4.6260, -4.4984],
         [-4.4181, -4.4961, -4.7563,  ..., -4.5908, -4.6278, -4.4993],
         [-4.4174, -4.4961, -4.7560,  ..., -4.5896, -4.6290, -4.4998]]],
       device='cuda:0', grad_fn=<SliceBackward0>)

In [ ]:
## split
# in_paragraphs = train[0]
# print(in_paragraphs)
# out_paragraphs = train[1]
# model.trainingLogits(in_paragraphs, out_paragraphs)

In [ ]:
# Tests the prediction methods.
batch = batch_generator.get_batch(2)
model.predictionStrings(batch[0].to(model.device), max_predicted_char=16)


['ºDqqqqqqqqqqqqqqq', 'ºDqqqqqqqqqqqqqqq']

In [ ]:
!pip install nltk
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def get_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1):
  # Tokenizing the reference and candidate texts
  reference_tok = [word_tokenize(reference[0])]
  candidate_tok = word_tokenize( candidate[0])
  return sentence_bleu(reference_tok, candidate_tok)


Training
==

In [ ]:
##grid search
for learning_rate in [0.02, 0.2]:
  for embedding_dim in [128,256,512]:
    for lstm_hidden_size in [256, 512, 1024]:
      for lstm_layers in [1,2,3]:
        for batch_size in [32, 64]:
          print("lr,emb,hid,lay,B : ", (learning_rate, embedding_dim, lstm_hidden_size , lstm_layers, batch_size))

          # model = Model(size_vocabulary=len(char_vocabulary), EOP_id=EOP_id, embedding_dim = 256, lstm_hidden_size=512, lstm_layers=1, device='cuda')
          model = Model(size_vocabulary=len(char_vocabulary), EOP_id=EOP_id, embedding_dim = embedding_dim, lstm_hidden_size=lstm_hidden_size, lstm_layers=lstm_layers, device='cuda')

          import time

          model.eval() # Tells Pytorch we are in evaluation/inference mode (can be useful if dropout is used, for instance).

          # Training procedure
          # learning_rate = 0.2
          learning_rate = learning_rate
          momentum = 0.99
          l2_reg = 0.0001
          optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=l2_reg)
          # Once the backward propagation has been done, call the 'step' method (with no argument) to update the parameters.
          # batch_size = 64 if(not use_toy_dataset) else 128
          batch_size = batch_size if(not use_toy_dataset) else 128
          subset = None # Use an integer to train on a smaller portion of the training set, otherwise use None.
          epoch_size = batch_generator.length() if(subset is None) else subset # In number of instances

          nb_epoch = 20 if(not use_toy_dataset) else 20
          epoch_id = 0 # Id of the current epoch
          instances_processed = 0 # Number of instances trained on in the current epoch
          epoch_loss = [] # Will contain the loss for each batch of the current epoch
          time_0 = time.time()
          while(epoch_id < nb_epoch):
            model.train() # Tells Pytorch we are in training mode (can be useful if dropout is used, for instance).

            model.zero_grad() # Makes sure the gradient is reinitialised to zero.

            batch = batch_generator.get_batch(batch_size, subset=subset)

            #print(ids_to_texts(batch[0])); print(ids_to_texts(batch[1]))
            in_paragraphs = batch[0].to(model.device)
            out_paragraphs = batch[1].to(model.device)

            # You have to (i) compute the prediction of the model, (ii) compute the loss, (iii) call "backward" on the loss and (iv) store the loss in "epoch_loss".
            # For the loss, use torch.nn.functional.nll_loss. Computes an average over all tokens of the batch, but do not take into account distribution logits that corresonds to padding characters. Read the documentation and be careful about the shape of your tensors.
            ###################

            logits = model.trainingLogits(in_paragraphs, out_paragraphs)
            # print(logits.shape)

            logits = logits.transpose(1, 2)

            pad_idx = batch_generator.padding_idx

            loss = torch.nn.functional.nll_loss(logits, out_paragraphs, reduction="mean", ignore_index=pad_idx)

            loss.backward()

            epoch_loss.append(loss.item())

            ###################

            optimizer.step() # Updates the parameters.

            instances_processed += batch_size
            if(instances_processed > epoch_size):
              print(f"-- END OF EPOCH {epoch_id}.")
              print(f"Average loss: {sum(epoch_loss) / len(epoch_loss)}.")
              duration = time.time() - time_0
              print(f"{duration} s elapsed (i.e. {duration / (epoch_id + 1)} s/epoch)")

              # Example of generation
              batch = batch_generator.get_batch(1, subset=subset)
              print(ids_to_texts(batch[0])) # Input paragraph
              generated = model.predictionStrings(batch[0].to(model.device), max_predicted_char=512) # Generated output paragraph.
              print(generated)

              ## BLEU
              print(f"BLEU : {round(get_bleu(ids_to_texts(batch[1]), generated),3)}")

              epoch_id += 1
              instances_processed -= epoch_size
              epoch_loss = []

lr,emb,hid,lay,B :  (0.02, 128, 256, 1, 32)
-- END OF EPOCH 0.
Average loss: 2.921983603898644.
27.10006546974182 s elapsed (i.e. 27.10006546974182 s/epoch)
["La mère Bovary, les jours suivants, fut très étonnée de la métamorphose de sa bru. En effet, Emma se montra plus docile, et même poussa la déférence jusqu'à lui demander une recette pour faire mariner des cornichons."]
['---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------']
BLEU : 0
-- END OF EPOCH 1.
Average loss: 2.191952940152616.
49.78644347190857 s elapsed (i.e. 24.893221

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

BLEU : 0.0
-- END OF EPOCH 2.
Average loss: 1.9717096096398878.
72.34357953071594 s elapsed (i.e. 24.114526510238647 s/epoch)
['-- Mais il y a peut-être à Rouen, dit le médecin, plusieurs demoiselles Lempereur qui sont maîtresses de piano?']
['-- Elle se partion de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de la comme de']
BLEU : 0.0
-- END OF EPOCH 3.
Average loss: 1.8551520474103032.
94.49635791778564 s elapsed (i.e. 23.62408947944641 s/epoch)
["Il allait chez eux, pour leur faire part d'un malheur. Le vent, la nuit dernière, avait jeté bas vingt pommiers d

KeyboardInterrupt: ignored

# New Section

In [ ]:
use_toy_dataset = True

In [ ]:
if(use_toy_dataset):
  prompt = ["AAAA. Now, please write 3 i." + EOP] #[paragraphs[0]]
  print(prompt[0])
  print(f"(Is this prompt in the training set? {prompt[0] in paragraphs})\n")

  for _ in range(10):
    batch = batch_generator.turn_into_batch(prompt)
    gen_texts = model.predictionStrings(batch.to(model.device), max_predicted_char=128)

    print(gen_texts[0])
    prompt = [gen_texts[0] + EOP]
else:
  prompt = ["Charles Bovary sortit une bonne bouteille de vin et alla chercher des verres pour ses invités."] * 10
  batch = batch_generator.turn_into_batch(prompt)
  gen_texts = model.predictionStrings(batch.to(model.device), max_predicted_char=1024)

  print(prompt[0])
  print()
  for i, gen_text in enumerate(gen_texts):
    print(f"{i}: ", end="")
    print(gen_text)

If your system does not work as expected, check that you are using a sensible loss function, but also check that your implementation matches the architecture depicted in https://moodle.u-paris.fr/mod/resource/view.php?id=648001.

If you cannot get your model to work even on the toy dataset, then there must be a bug somewhere.

Read the remarks at the beginning of the TP again.

Once you are sure that your system is correctly implemented and generates texts that look a little bit like natural language, find ways to improve the system.
Here are some ideas (ordered arbitrarily):

*   Compute a measure that evaluates the performance of the model.
*   Split your dataset into a training and a development section, and use this split in a relevant way.
*   Implement beam decoding instead of greedy decoding.
*   Use other units of text instead of characters (ex: words, word-pieces).
*   Add more data to the dataset.
*   Use graphs to visualise the training process and the predictions.

Document in a text cell all of the changes that you make to the system and describe their impact (qualitatively **and** quantitatively).